In [1]:
import requests
import pandas as pd
from datetime import datetime

# URL de base de l'API de prévisions Open-Meteo
BASE_URL = "https://api.open-meteo.com/v1/forecast"

def get_precipitation_forecast(latitude, longitude, start_date, end_date):
    """
    Récupère les données de prévisions quotidiennes de précipitations via l'API Open-Meteo.
    """
    try:
        # Paramètres de la requête API
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "daily": ["precipitation_sum", "precipitation_probability_max"],
            "timezone": "auto",  # Définir le fuseau horaire automatiquement
            "start_date": start_date,
            "end_date": end_date
        }

        print(f"\nRequête pour Lat: {latitude}, Lon: {longitude} du {start_date} au {end_date}...")
        
        # Envoi de la requête GET
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Lance une exception pour les codes d'état HTTP non réussis

        data = response.json()
        
        # Vérification si les données de prévisions quotidiennes sont présentes
        if "daily" not in data:
            print("Erreur: L'API n'a pas retourné de données quotidiennes (vérifiez les dates, la limite est de 16 jours pour les prévisions futures).")
            return None

        daily_data = data["daily"]
        
        # Création d'un DataFrame Pandas pour une analyse facile
        df = pd.DataFrame({
            "Date": daily_data["time"],
            "Precipitation_mm": daily_data["precipitation_sum"],
            "Probabilite_Precipitation_Max": daily_data["precipitation_probability_max"]
        })
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à l'API : {e}")
        return None
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {e}")
        return None

def main():
    """
    Fonction principale pour l'interaction utilisateur.
    """
    print("="*50)
    print(" PRÉVISION DE PRÉCIPITATIONS AVEC OPEN-METEO ")
    print(" (Prévisions limitées à 16 jours à partir d'aujourd'hui) ")
    print("="*50)
    
    # 1. Demande des coordonnées
    try:
        latitude = float(input("Entrez la Latitude (ex: 48.8566 pour Paris) : "))
        longitude = float(input("Entrez la Longitude (ex: 2.3522 pour Paris) : "))
    except ValueError:
        print("Erreur : Veuillez entrer des nombres valides pour la latitude et la longitude.")
        return

    # 2. Demande de la période
    print("\nFormat de date requis : AAAA-MM-JJ (ex: 2025-10-15)")
    try:
        start_date_str = input("Entrez la date de DÉBUT de la prévision : ")
        end_date_str = input("Entrez la date de FIN de la prévision : ")
        
        # Validation du format de date
        datetime.strptime(start_date_str, '%Y-%m-%d')
        datetime.strptime(end_date_str, '%Y-%m-%d')
        
    except ValueError:
        print("Erreur : Format de date incorrect. Utilisez AAAA-MM-JJ.")
        return

    # 3. Récupération des données
    forecast_data = get_precipitation_forecast(latitude, longitude, start_date_str, end_date_str)

    if forecast_data is not None and not forecast_data.empty:
        
        print("\n" + "="*50)
        print(f"RÉSUMÉ DES PRÉVISIONS ({start_date_str} à {end_date_str})")
        print("="*50)
        
        # Calcul des statistiques agrégées
        total_days = len(forecast_data)
        total_precipitation = forecast_data["Precipitation_mm"].sum()
        
        # Nombre de jours avec prévisions de pluie (> 0 mm)
        rainy_days_count = (forecast_data["Precipitation_mm"] > 0).sum()
        
        # Probabilité de précipitation moyenne et max
        avg_prob = forecast_data["Probabilite_Precipitation_Max"].mean()
        
        print(f"Jours analysés : {total_days}")
        print(f"Précipitation totale prévue (somme) : {total_precipitation:.2f} mm 🌧️")
        print(f"Nombre de jours avec pluie prévue : {rainy_days_count} / {total_days}")
        print(f"Probabilité maximale de précipitation moyenne : {avg_prob:.1f} %")
        print("-" * 50)
        
        # Affichage des données détaillées
        print("\nDétails Jour par Jour :")
        print(forecast_data.to_string(index=False))

    elif forecast_data is not None:
         print("\nAucune donnée de prévision disponible pour cette période.")

if __name__ == "__main__":
    main()

 PRÉVISION DE PRÉCIPITATIONS AVEC OPEN-METEO 
 (Prévisions limitées à 16 jours à partir d'aujourd'hui) 

Format de date requis : AAAA-MM-JJ (ex: 2025-10-15)

Requête pour Lat: 53.0, Lon: 26.0 du 2025-10-01 au 2025-10-10...

RÉSUMÉ DES PRÉVISIONS (2025-10-01 à 2025-10-10)
Jours analysés : 10
Précipitation totale prévue (somme) : 7.30 mm 🌧️
Nombre de jours avec pluie prévue : 5 / 10
Probabilité maximale de précipitation moyenne : 24.3 %
--------------------------------------------------

Détails Jour par Jour :
      Date  Precipitation_mm  Probabilite_Precipitation_Max
2025-10-01               0.0                              6
2025-10-02               0.1                             22
2025-10-03               0.0                              8
2025-10-04               0.0                             51
2025-10-05               1.0                             80
2025-10-06               0.0                              4
2025-10-07               0.4                              8
2025-